# Analyze robustness of  policies

The code and explainations are based on Assignment 8 (Jan Kwakkel), modified to import experiment results and change plot format


In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ema_workbench.analysis import parcoords

# Change plot format (increase size and font size
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['figure.dpi'] = 200
sns.set(font_scale=1.5)
#plt.rcParams["font.family"] = "Times New Roman"
#plt.rcParams["font.size"] = "50"

from ema_workbench import (Model, RealParameter, ScalarOutcome, load_results)

# Import model
from problem_formulation import get_model_for_problem_formulation

# Get model instance
model, steps = get_model_for_problem_formulation(3)
#outcomes = model.outcomes

For this, the experiments that were executed for the policies resulting from MORDM are imported and analyzed.

In [ ]:
# Load experiments performed for policies
fn = 'SECOND candidate solutions MORDM 1000scenarios.tar.gz'

try:
    # Store results in dataframe
    results = load_results(fn)
except IOError:
    print("Error loading data")


experiments, outcomes = results
experiments


Now, we  evaluate the **robustness** of each of the policy options based on these scenario results.
We can calculate the robustness of a policy option in terms of its performance on an outcome indicator across the 1000 scenarios. In other words, we can identify how robust a policy is in terms of each outcome indicator, and investigate the robustness tradeoffs.

**The following function calculates the signal-to-noise ratio for the outcome indicators.**

In [ ]:
def s_to_n(data, direction):
    mean = np.mean(data)
    std = np.std(data)
    
    if direction==ScalarOutcome.MAXIMIZE:
        return mean/std
    else:
        return mean*std

The signal to noise ratio is calculated by iterating over the policies.
Next, we iterate over the outcomes. For each outcome, we retrieve the results associated with the current policy. We than can calculate the signal to noise ratio and store it in the scores dictionary and convert the dictionary to a dataframe lateron.

In [ ]:
overall_scores = {}
for policy in np.unique(experiments['policy']):
    scores = {}
    
    logical = experiments['policy']==policy
    
    for outcome in model.outcomes:
        value  = outcomes[outcome.name][logical]
        sn_ratio = s_to_n(value, outcome.kind)
        scores[outcome.name] = sn_ratio
    overall_scores[policy] = scores
scores = pd.DataFrame.from_dict(overall_scores).T
scores

# Signal to noise ratio

In [ ]:
colors = sns.color_palette()

data = scores

In [ ]:

# makes it easier to identify the policy associated with each line
# in the parcoords plot
# data['policy'] = data.index.astype("float64")

# Create dictionary for axis format, use scientific notation for better readiblity 
formatMinMax = {}

for outcome in model.outcomes:
    formatMinMax[outcome.name] = ".2e"

limits = parcoords.get_limits(data)
#limits.loc[0, ['utility', 'inertia', 'reliability', 'max_P']] = 0

paraxes = parcoords.ParallelAxes(limits, formatter=formatMinMax, fontsize=20)
for i, (index, row) in enumerate(data.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(index), color=colors[i])
paraxes.legend()

#for outcome in model.outcomes:
#    if(outcome.kind == -1):
#        paraxes.invert_axis(outcome.name)

fig = plt.figure(figsize=(30,30))
plt.show()

In [ ]:
scores = pd.DataFrame(scores)
sns.heatmap(scores/scores.max(), cmap='viridis', annot=True)
plt.show()

An ideal solution has a low signal to noise ratio for all outcomes of interest because all outcomes are to be minimized.

# Maximum regret

Another robustness metric is **maximum regret**, calculated again for each policy and for each outcome indicator. *Regret* is defined for each policy under each scenario, as the difference between the performance of the policy in a specific scenario and the berformance of a no-regret (i.e. best possible result in that scenario) or reference policy. The *maximum regret*  is then the maximum of such regret values across all scenarios. We of course favor policy options with low *maximum regret* values. 

In [ ]:
def calculate_regret(data, best):
    return np.abs(best-data)

In [ ]:
experiments, outcomes = results

overall_regret = {}
max_regret = {}
for outcome in model.outcomes:
    policy_column = experiments['policy']
    
    # create a DataFrame with all the relevant information
    # i.e., policy, scenario_id, and scores
    data = pd.DataFrame({outcome.name: outcomes[outcome.name], 
                         "policy":experiments['policy'],
                         "scenario":experiments['scenario']})
    
    # reorient the data by indexing with policy and scenario id
    data = data.pivot(index='scenario', columns='policy')
    
    # flatten the resulting hierarchical index resulting from 
    # pivoting, (might be a nicer solution possible)
    data.columns = data.columns.get_level_values(1)
    
    outcome_regret = (data.max(axis=1)[:, np.newaxis] - data).abs()
    
    overall_regret[outcome.name] = outcome_regret
    max_regret[outcome.name] = outcome_regret.max()
    

In [ ]:
max_regret = pd.DataFrame(max_regret)
sns.heatmap(max_regret/max_regret.max(), cmap='viridis', annot=True)
plt.show()

In [ ]:
colors = sns.color_palette()

data = max_regret

# makes it easier to identify the policy associated with each line
# in the parcoords plot
# data['policy'] = data.index.astype("float64")

limits = parcoords.get_limits(data)
#limits.loc[0, ['utility', 'inertia', 'reliability', 'max_P']] = 0

paraxes = parcoords.ParallelAxes(limits, formatter=formatMinMax, fontsize=20)
for i, (index, row) in enumerate(data.iterrows()):
    paraxes.plot(row.to_frame().T, label=str(index), color=colors[i])
paraxes.legend()
    
plt.show()

We can see that the robustness differs for every outcome indicator. 

In [ ]:
from collections import defaultdict

policy_regret = defaultdict(dict)
for key, value in overall_regret.items():
    for policy in value:
        policy_regret[policy][key] = value[policy]

In [ ]:
# this generates a 2 by 2 axes grid, with a shared X and Y axis
# accross all plots
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(10,10), 
                         sharey=True, sharex=True)

# to ensure easy iteration over the axes grid, we turn it
# into a list. Because there are four plots, I hard coded
# this. 
axes = [axes[0,0], axes[0,1],
        axes[1,0],]

# zip allows us to zip together the list of axes and the list of 
# key value pairs return by items. If we iterate over this
# it returns a tuple of length 2. The first item is the ax
# the second items is the key value pair.
for ax, (policy, regret) in zip(axes, policy_regret.items()):
    data = pd.DataFrame(regret)

    # we need to scale the regret to ensure fair visual
    # comparison. We can do that by divding by the maximum regret
    data = data/max_regret.max(axis=0)
    sns.boxplot(data=data, ax=ax)
    
    # removes top and left hand black outline of axes
    sns.despine()
    
    # ensure we know which policy the figure is for
    ax.set_title(str(policy))
plt.show()

This is in line with the maximum regret parallel coordinates plot, but we get some more details.

We now have an understanding of which solutions have decent robustness using 2 different robustness metrics. 

A related but different question is to assess the uncertain conditions under which we get poor performance. For this, we can use scenario discovery. Since we want to identify the uncertainties only, we can remove the policy and lever columns from the experiments DataFrame. 

**Perform Scenario Discovery, focussed on understanding the conditions under which utility is lower than 0.35**

In [ ]:
from ema_workbench.analysis import prim

x = experiments.drop(columns=['policy', 'c1','c2', 'r1', 'r2', 'w1'])
y = outcomes['utility'] < 0.35

prim_alg = prim.Prim(x,y, threshold=0.5)
box = prim_alg.find_box()

In [ ]:
box.inspect_tradeoff()

the choice for box 42 is somewhat arbitrary. 

In [ ]:
box.inspect(42)

In [ ]:
box.select(42)

In [ ]:
scens_in_box = experiments.iloc[box.yi]

In [ ]:
outcomes_in_box = {k:v[box.yi] for k,v in outcomes.items()}

